# 패키지 설치
- HuggingFace는 NLP분야에서 BERT 모델 같은 트랜스포머 모델들을 쉽게 다룰 수 있게 해주는 패키지입니다.
- 기본적으로 pytorch 기반으로 만들어져 있지만, 텐서플로우 2.0에서도 사용 가능합니다.



In [1]:
 # huggingface 패키지를 Colab에 설치
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.2 MB/s eta 0:00:00


## 토크나이저 초기화

BERT(`kcbert-base`) 모델이 쓰는 토크나이저를 선언합니다.

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(
    "beomi/kcbert-base",
    do_lower_case=False,
)

## 모델 초기화

BERT(`kcbert-base`) 모델을 읽어들입니다.

In [3]:
from transformers import BertConfig, BertModel

pretrained_model_config = BertConfig.from_pretrained(
    "beomi/kcbert-base"
)

model = BertModel.from_pretrained(
    "beomi/kcbert-base",
    config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


`pretrained_model_config`의 내용을 확인합니다.

In [4]:
pretrained_model_config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

## 모델 입력값 만들기

문장을 모델 입력값으로 만들어보겠습니다.

In [11]:
sentences = "안녕하세요 만나서 반갑습니다!!"
features = tokenizer( # 토큰화
    sentences,
    max_length=15, # 최대 길이
    padding="max_length", # 패딩 방식
    truncation=True, # 최대 길이보다 길면 자르기
)

`features`의 내용을 확인합니다.

In [12]:
features.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [13]:
features['input_ids'] # 토큰화된 문장

[2, 19017, 8482, 14693, 1483, 4981, 8046, 5, 5, 3, 0, 0, 0, 0, 0]

In [14]:
features['attention_mask'] # 패딩된 부분을 0으로 표시

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

In [15]:
features['token_type_ids'] # 0(sentence A)과 1(sentence B) 문장 구분

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [16]:
tokenizer.convert_ids_to_tokens(features['input_ids']) # 토큰화된 문장을 다시 텍스트로 변환

['[CLS]',
 '안녕',
 '##하세요',
 '만나서',
 '반',
 '##갑',
 '##습니다',
 '!',
 '!',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [19]:
tokenizer.decode(features['input_ids']) # 토큰화된 문장을 다시 텍스트로 변환

'[CLS] 안녕하세요 만나서 반갑습니다!! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]'

### 두 문장을 넣어봅시다!
- List[List[str]] 형태로 입력이 들어오면 tokenizer는 하위 list(List[str])가 sentence A, sentence B를 입력하는 것이라 이해합니다.

In [57]:
sentences = [["안녕하세요 만나서 반갑습니다!!","SK는 밥이 맛있는 것 같아요!!"]]

features = tokenizer(
    sentences,
    max_length=20,
    padding="max_length",
    truncation=True,
)

### features들을 확인해봅시다!

In [30]:
features['attention_mask'] # 패딩된 부분을 0으로 표시

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

### 토큰화된 문장을 다시 텍스트로 변환해봅시다!

In [31]:
features['token_type_ids'] # 0(sentence A)과 1(sentence B) 문장 구분

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [32]:
tokenizer.convert_ids_to_tokens(features['input_ids'][0]) # 토큰화된 문장을 다시 텍스트로 변환

['[CLS]',
 '안녕',
 '##하세요',
 '만나서',
 '반',
 '##갑',
 '##습니다',
 '!',
 '!',
 '[SEP]',
 'SK',
 '##는',
 '밥이',
 '맛',
 '##있는',
 '것',
 '같아요',
 '!',
 '!',
 '[SEP]']

In [58]:
tokenizer.batch_decode(features['input_ids']) # 토큰화된 문장을 다시 텍스트로 변환

['[CLS] 안녕하세요 만나서 반갑습니다!! [SEP] SK는 밥이 맛있는 것 같아요!! [SEP]']

## BERT 임베딩 추출

위에서 만든 `features`를 파이토치 텐서(tensor)로 변환합니다.

In [59]:
import torch
features = {k: torch.tensor(v) for k, v in features.items()} # 텐서로 변환

BERT 모델에 `features`를 입력해 계산합니다.

In [60]:
outputs = model(**features) # 모델에 입력, 딕셔너리 형태의 `features`의 키-값을 함수의 인자로 전달
# `model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)`처럼 작동

BERT 마지막 레이어의 단어 수준 벡터들을 확인합니다.

In [61]:
# 마지막 레이어의 hidden_state : (batch_size, sequence_length, hidden_size)
outputs.last_hidden_state.size()

torch.Size([1, 20, 768])

BERT 마지막 레이어의 문서 수준 벡터를 확인합니다.

In [62]:
# 마지막 hidden_state의 첫번째 토큰인 [CLS] 토큰의 embedding을 반환
outputs.pooler_output.size()

torch.Size([1, 768])